### Types of Restuarants around the Perth Metropolitan Area

In [1]:
#dependencies et al.
import matplotlib.pyplot as plt
import json
import requests
import pandas as pd
import numpy as np
import time

#api_keys
import api_keys

#central point of Perth CBD to define North and South
central_lat = -32.0391738
central_lng = 115.6813559


In [2]:
#bring in perth_metro_house_prices.csv
load_csv = "../data/perth_metro_house_prices.csv"
perth_house_prices = pd.read_csv(load_csv)
perth_house_prices.head()

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,1 Acorn Place,South Lake,565000,4,2,2.0,600,160,2003.0,18300,Cockburn Central Station,1800,09-2018\r,6164,-32.115900,115.842450,LAKELAND SENIOR HIGH SCHOOL,0.828339,NaN
1,1 Addis Way,Wandi,365000,3,2,2.0,351,139,2013.0,26900,Kwinana Station,4900,02-2019\r,6167,-32.193470,115.859553,ATWELL COLLEGE,5.524324,129.0
2,1 Ainsley Court,Camillo,287000,3,1,1.0,719,86,1979.0,22600,Challis Station,1900,06-2015\r,6111,-32.120578,115.993579,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,113.0
3,1 Albert Street,Bellevue,255000,2,1,2.0,651,59,1953.0,17900,Midland Station,3600,07-2018\r,6056,-31.900547,116.038009,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,NaN
4,1 Aman Place,Lockridge,325000,4,1,2.0,466,131,1998.0,11200,Bassendean Station,2000,11-2016\r,6054,-31.885790,115.947780,KIARA COLLEGE,1.514922,NaN


In [3]:
#create north_south column to distinguish suburb location vs perth CBD as central point
north_south = []
for row in perth_house_prices['LATITUDE']:
    if row >= central_lat:north_south.append('NORTH')
    else:
        north_south.append('SOUTH')
        
perth_house_prices.insert(2, 'NORTH_SOUTH', north_south)
perth_house_prices.head()

,ADDRESS,SUBURB,NORTH_SOUTH,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,1 Acorn Place,South Lake,SOUTH,565000,4,2,2.0,600,160,2003.0,18300,Cockburn Central Station,1800,09-2018\r,6164,-32.115900,115.842450,LAKELAND SENIOR HIGH SCHOOL,0.828339,NaN
1,1 Addis Way,Wandi,SOUTH,365000,3,2,2.0,351,139,2013.0,26900,Kwinana Station,4900,02-2019\r,6167,-32.193470,115.859553,ATWELL COLLEGE,5.524324,129.0
2,1 Ainsley Court,Camillo,SOUTH,287000,3,1,1.0,719,86,1979.0,22600,Challis Station,1900,06-2015\r,6111,-32.120578,115.993579,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,113.0
3,1 Albert Street,Bellevue,NORTH,255000,2,1,2.0,651,59,1953.0,17900,Midland Station,3600,07-2018\r,6056,-31.900547,116.038009,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,NaN
4,1 Aman Place,Lockridge,NORTH,325000,4,1,2.0,466,131,1998.0,11200,Bassendean Station,2000,11-2016\r,6054,-31.885790,115.947780,KIARA COLLEGE,1.514922,NaN


In [4]:
#create a smaller table to generate a groupby object to determine median house price by postcode and north / south.
median_house_price_postcode = perth_house_prices[['POSTCODE',
                                                  'SUBURB',
                                                  'NORTH_SOUTH',
                                                  'PRICE']]
median_house_price_postcode.head()

,POSTCODE,SUBURB,NORTH_SOUTH,PRICE
0,6164,South Lake,SOUTH,565000
1,6167,Wandi,SOUTH,365000
2,6111,Camillo,SOUTH,287000
3,6056,Bellevue,NORTH,255000
4,6054,Lockridge,NORTH,325000


In [5]:
#groupby on postcode
grouped_median_house_price_postcode = median_house_price_postcode.groupby(['POSTCODE','SUBURB'])
median_price_by_postcode_mean = grouped_median_house_price_postcode['PRICE'].mean()
median_price_by_postcode = pd.DataFrame({"Median House Price":round(median_price_by_postcode_mean,0)})
median_price_by_postcode.reset_index(inplace = True)
median_price_by_postcode.head()

,POSTCODE,SUBURB,Median House Price
0,6003,Highgate,790883.0
1,6003,Northbridge,721636.0
2,6004,East Perth,510182.0
3,6005,West Perth,681065.0
4,6006,North Perth,849587.0


In [6]:
#groupby on north_south
grouped_median_house_price_north_south = median_house_price_postcode.groupby(['NORTH_SOUTH'])
median_price_by_north_south_mean = grouped_median_house_price_north_south['PRICE'].mean()
median_price_by_north_south = pd.DataFrame({"Median House Price":round(median_price_by_north_south_mean,0)})
median_price_by_north_south.head()

,Median House Price
NORTH_SOUTH,
NORTH,683849.0
SOUTH,532993.0


In [7]:
#bring in consolidated_census_data.csv for socio-demographic information by SSC_CODE - 
#used excel lookups to determine suburbs and postcodes and cross referenced against census metadata.
load_csv = "../data/consolidated_census_data.csv"
census_data = pd.read_csv(load_csv)
census_data.head()

,SSC_CODE_2016,Suburb,Suburb (Original),Postcode,Suburb sqkm,Median Age (Persons),Median Mortgage Repayments (Monthly),Median Total Personal Income (Weekly),Median Rent (Weekly),Median Total Family Income (Weekly),...,People (15-24 year),People (25-34 year),People (35-44 year),People (45-54 year),People (55-64 year),People (65-74 year),People (75-84 year),People (85 and over year),Total Number of People,Unemployment Rate (%)
0,SSC50001,Abba River,Abba River,6280,28.5156,35,2167,675,220,1583,...,7,5,18,10,5,4,0,0,66,0.0
1,SSC50002,Abbey,Abbey,6280,2.6436,39,1950,665,400,1730,...,112,125,163,174,144,119,56,14,1210,5.0
2,SSC50003,Acton Park,Acton Park (WA),6280,24.8542,40,1617,756,185,1843,...,10,14,12,12,16,7,4,0,100,8.0
3,SSC50004,Adamsvale,Adamsvale,6375,246.5202,43,0,1625,0,3499,...,0,5,7,4,5,3,0,0,31,0.0
4,SSC50005,Ajana,Ajana,6532,727.8242,37,0,478,40,1312,...,8,10,6,14,42,51,14,0,164,0.0


In [8]:
#remove unecessary columns
census_data = census_data.drop(['SSC_CODE_2016','Suburb (Original)'], axis = 1)
#show columns available in census_data
list(census_data.columns.values)

['Suburb',
 'Postcode',
 'Suburb sqkm',
 'Median Age (Persons)',
 'Median Mortgage Repayments (Monthly)',
 'Median Total Personal Income (Weekly)',
 'Median Rent (Weekly)',
 'Median Total Family Income (Weekly)',
 'Average Number of Persons per Bedroom',
 'Median Total Household Income (Weekly)',
 'Average Household Size',
 'People (0-14 year)',
 'People (15-24 year)',
 'People (25-34 year)',
 'People (35-44 year)',
 'People (45-54 year)',
 'People (55-64 year)',
 'People (65-74 year)',
 'People (75-84 year)',
 'People (85 and over year)',
 'Total Number of People',
 'Unemployment Rate (%)']

In [9]:
median_price_by_postcode.dtypes

POSTCODE                int64
SUBURB                 object
Median House Price    float64
dtype: object

In [10]:
median_price_by_postcode['Median House Price'] = median_price_by_postcode['Median House Price'].astype(int)
median_price_by_postcode.dtypes

POSTCODE               int64
SUBURB                object
Median House Price     int32
dtype: object

In [11]:
census_data.dtypes

Suburb                                     object
Postcode                                    int64
Suburb sqkm                               float64
Median Age (Persons)                        int64
Median Mortgage Repayments (Monthly)        int64
Median Total Personal Income (Weekly)       int64
Median Rent (Weekly)                        int64
Median Total Family Income (Weekly)         int64
Average Number of Persons per Bedroom     float64
Median Total Household Income (Weekly)      int64
Average Household Size                    float64
People (0-14 year)                          int64
People (15-24 year)                         int64
People (25-34 year)                         int64
People (35-44 year)                         int64
People (45-54 year)                         int64
People (55-64 year)                         int64
People (65-74 year)                         int64
People (75-84 year)                         int64
People (85 and over year)                   int64


In [12]:
#merge census data with median_price_by_postcode
pc_price_census_data = census_data.merge(right = median_price_by_postcode, 
                                         how = 'right', 
                                         left_on = 'Postcode', 
                                         right_on = 'POSTCODE')

pc_price_census_data.head()

,Suburb,Postcode,Suburb sqkm,Median Age (Persons),Median Mortgage Repayments (Monthly),Median Total Personal Income (Weekly),Median Rent (Weekly),Median Total Family Income (Weekly),Average Number of Persons per Bedroom,Median Total Household Income (Weekly),...,People (45-54 year),People (55-64 year),People (65-74 year),People (75-84 year),People (85 and over year),Total Number of People,Unemployment Rate (%),POSTCODE,SUBURB,Median House Price
0,Alexander Heights,6064,3.1816,37,1800,625,390,1704,0.8,1590,...,1163,1056,515,281,64,7844,8.8,6064,Alexander Heights,453285
1,Girrawheen,6064,4.0937,34,1549,489,320,1208,0.9,1104,...,1063,886,706,306,68,8676,12.7,6064,Alexander Heights,453285
2,Koondoola,6064,3.5198,33,1400,443,310,1077,0.9,1009,...,480,395,341,142,28,3977,15.6,6064,Alexander Heights,453285
3,Marangaroo,6064,4.7933,36,1757,548,369,1534,0.8,1419,...,1428,1191,797,422,217,10476,9.5,6064,Alexander Heights,453285
4,Alexander Heights,6064,3.1816,37,1800,625,390,1704,0.8,1590,...,1163,1056,515,281,64,7844,8.8,6064,Girrawheen,318998


In [13]:
pc_price_census_data = pc_price_census_data.drop(['POSTCODE','SUBURB'], axis = 1)
pc_price_census_data = pc_price_census_data[['Postcode',
                                            'Suburb',
                                            'Suburb sqkm',
                                            'Median House Price',
                                            'Median Age (Persons)',
                                            'Median Mortgage Repayments (Monthly)',
                                            'Median Total Personal Income (Weekly)',
                                            'Median Rent (Weekly)',
                                            'Median Total Family Income (Weekly)',
                                            'Average Number of Persons per Bedroom',
                                            'Median Total Household Income (Weekly)',
                                            'Average Household Size',
                                            'People (0-14 year)',
                                            'People (15-24 year)',
                                            'People (25-34 year)',
                                            'People (35-44 year)',
                                            'People (45-54 year)',
                                            'People (55-64 year)',
                                            'People (65-74 year)',
                                            'People (75-84 year)',
                                            'People (85 and over year)',
                                            'Total Number of People',
                                            'Unemployment Rate (%)']]
list(pc_price_census_data.columns.values)

['Postcode',
 'Suburb',
 'Suburb sqkm',
 'Median House Price',
 'Median Age (Persons)',
 'Median Mortgage Repayments (Monthly)',
 'Median Total Personal Income (Weekly)',
 'Median Rent (Weekly)',
 'Median Total Family Income (Weekly)',
 'Average Number of Persons per Bedroom',
 'Median Total Household Income (Weekly)',
 'Average Household Size',
 'People (0-14 year)',
 'People (15-24 year)',
 'People (25-34 year)',
 'People (35-44 year)',
 'People (45-54 year)',
 'People (55-64 year)',
 'People (65-74 year)',
 'People (75-84 year)',
 'People (85 and over year)',
 'Total Number of People',
 'Unemployment Rate (%)']

In [14]:
pc_price_census_data.to_csv("../Outputs/census_data_post_median_house_price.csv", index=False)

In [15]:
#bring in wa_electorial_populations.csv
load_csv = "../data/wa_electorial_populations.csv"
wa_populations = pd.read_csv(load_csv)
wa_populations.head()

,Electorial_Group,Suburb,2020_Population
0,50106,Albany (South West),38295
1,50202,Armadale (East Metropolitan),50268
2,50304,Balcatta (North Metropolitan),44478
3,50405,Baldivis (South Metropolitan),60243
4,50502,Bassendean (East Metropolitan),44134
